In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/mnt/xfs/home/bencw/src/context-cite


/mnt/xfs/home/bencw/conda_envs/cite/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import numpy as np
import torch as ch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
from context_cite import ContextCiter

[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/xfs/home/bencw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=ch.float16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
context = "On April 8, 2024, a total solar eclipse will be visible in the continental United States. Glimpse the full glory of the Sun's outer atmosphere, or corona, visible only when the Sun's disc is completely covered by the moon. To witness this incredible total solar eclipse, you will need to be within the 115-mile-wide path of totality. The path arches from Mexico to Texas to Maine. Unless you're on that line—the path of totality—you'll only see a partial eclipse. The difference between a total solar eclipse and a partial one is literally the difference between night and day, so get yourself into the path of totality if you can! The closer you are to the center of that path, the longer totality, the maximum point of the eclipse, will last. The whole event will take about two and a half hours, but totality will only last about four minutes. Most towns in the US aren't on the path. If you stay home, you'll likely only see a partial eclipse. Make your travel plans early as hotels and campsites usually book up months in advance! The shadow of the moon goes through several time zones. Check the local time to ensure you don't miss the total phase of the solar eclipse, which only lasts about four minutes. To determine the precise start time, end time, and duration of totality for your exact location on eclipse day, use this interactive eclipse map developed by timeanddate.com. Find a sunny location along the path, or the clouds will hide the Sun before the moon does. According to predictions, your best bets for clear skies are in central Mexico and south-central Texas. You will need eclipse glasses to safely view the Sun. We also highly recommend that you gather up colanders, slotted spoons, or anything else with small holes (you can poke holes in a piece of cardboard), because the eclipse will make very cool shadows. Check out our safe-viewing page for tips on building other kinds of viewers."
query = "I live in Boston. Where and when should I go to see the eclipse? Please answer briefly."

In [6]:
cc = ContextCiter(model, tokenizer, context, query)

In [7]:
cc.response

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" To see the total solar eclipse in Boston on April 8, 2024, you will need to travel to the path of totality, which passes through Maine. The closest town in Maine with totality is Madawaska, near the Canadian border. The eclipse will begin around 2:38 PM EDT and last approximately 4 minutes. Make travel plans early, as accommodations may book up months in advance. Remember to bring eclipse glasses for safe viewing and materials for observing the eclipse's unique shadows."

In [8]:
cc.response_metadata

,start_index,end_index,sentence
0,0,136,To see the total solar eclipse in Boston on A...
1,137,216,The closest town in Maine with totality is Mad...
2,217,292,The eclipse will begin around 2:38 PM EDT and ...
3,293,366,"Make travel plans early, as accommodations may..."
4,367,475,Remember to bring eclipse glasses for safe vie...


In [9]:
cc.get_scores(0, 136)

  0%|          | 0/256 [00:00<?, ?it/s]

array([16.48157988,  0.65075504,  3.66754446,  8.49358765,  0.323622  ,
       -0.        ,  1.24949428, -0.18974822,  1.29665971,  0.17359111,
        0.10932293,  0.40512605,  0.25062863,  1.55528302,  1.10721519,
       -7.19719984, -0.        ,  0.247937  , -0.        ])

In [10]:
cc.show_attributions(0, 136)

,Score,Source
0,16.482,"On April 8, 2024, a total solar eclipse will be visible in the continental United States."
1,8.494,The path arches from Mexico to Texas to Maine.
2,3.668,"To witness this incredible total solar eclipse, you will need to be within the 115-mile-wide path of totality."
3,1.555,"To determine the precise start time, end time, and duration of totality for your exact location on eclipse day, use this interactive eclipse map developed by timeanddate.com."
4,1.297,Most towns in the US aren't on the path.
5,1.249,"The closer you are to the center of that path, the longer totality, the maximum point of the eclipse, will last."
6,1.107,"Find a sunny location along the path, or the clouds will hide the Sun before the moon does."
7,0.651,"Glimpse the full glory of the Sun's outer atmosphere, or corona, visible only when the Sun's disc is completely covered by the moon."
8,0.405,The shadow of the moon goes through several time zones.
9,0.324,Unless you're on that line—the path of totality—you'll only see a partial eclipse.
